# Example pipeline execution

Here is an example of what it takes to configure and run a pipeline with flat-fielding and background subtraction.

## Setup CRDS

Make sure you have a local checkout of the CRDS cache as explained in the [Getting started page](getting-started.rst). Run the `setup_local_crds.sh` to setup the enviroment variables needed to point the crds software to the CRDS cache. Optionally source this in your shell configuration to automatically set this up.

## Get input simulations
Download simulated input FITS files for the IRIS imager from [Figshare](https://figshare.com/articles/TMT_IRIS_test_simulations/9941939). It contains a raw science frame, a raw flat frame and a raw background frame.

In [ ]:
import urllib.request
import zipfile
import os.path

if not os.path.exists("iris_example_data/raw_flat_frame_cal.fits"):
    urllib.request.urlretrieve(
        "https://ndownloader.figshare.com/articles/9941939/versions/1",
        "iris_example_data.zip"
    )

    with zipfile.ZipFile("iris_example_data.zip", 'r') as zip_ref:
        zip_ref.extractall("iris_example_data")

# this could take a few minutes

In [ ]:
%ls iris_example_data/

## Preprocess the flat frame
First we need to remove the dark frame from the flat frame and normalize it. A dark frame is already available in the CRDS and the pipeline knows how to retrieve it based on the metadata in the FITS file headers.

We can check in the package documentation what are the available pipelines and check the configuration options of the pipeline.ProcessFlatfieldL2 class.

We do not need to customize it so we can directly call it from tmtrun and pass the input FITS file:

This will pickup the relevant dark frame from the CRDS and process the file:

In [ ]:
!tmtrun iris_pipeline.pipeline.ProcessFlatfieldL2 iris_example_data/raw_flat_frame_cal.fits

## Configure the image processing pipeline

The [`ProcessImagerL2Pipeline`](api/iris_pipeline.pipeline.ProcessImagerL2Pipeline.rst#iris_pipeline.ProcessImagerL2Pipeline) can be configured using a INI-style configuration file:

In [ ]:
%%file image2_iris.cfg


name = "ProcessImagerL2Pipeline"
class = "iris_pipeline.pipeline.ProcessImagerL2Pipeline"
save_results = True

    [steps]
      [[bkg_subtract]]
      [[assign_wcs]]
        skip = True
      [[flat_field]]
        config_file = flat_field.cfg
      [[photom]]
        skip = True
      [[resample]]
        skip = True

first we specify that we want to execute the pipeline defined in the pipeline, then we can configure each of the steps, for example skip some of them. Also we can import the configuration of a step from another file, in this case flat_field.cfg:

In [ ]:
%%file flat_field.cfg

name = "flat_field"
class = "jwst.flatfield.FlatFieldStep"
override_flat = 'raw_flat_frame_flat.fits'

## Define the input data

JWST created a specification for defining how input files should be used by a pipeline, it is a JSON file named an association, see the [JWST documentation](https://jwst-pipeline.readthedocs.io/en/latest/jwst/associations/overview.html).

In our example we need to specify a input raw science frame ad a background to be subtracted:

In [ ]:
%%file association.json

{
    "asn_rule": "Asn_Lv2Image",
    "asn_pool": "pool",
    "asn_type": "image2",
    "products": [
        {
            "name": "test_iris_subtract_bg_flat",
            "members": [
                {
                    "expname": "iris_example_data/raw_science_frame_sci.fits",
                    "exptype": "science"
                },
                {
                    "expname": "iris_example_data/raw_background_frame_cal.fits",
                    "exptype": "background"
                }
            ]
        }
    ]
}

## Execute the pipeline from the command line

Execute the pipeline from the command line
We can use tmtrun from a terminal to execute the pipeline:

In [ ]:
!tmtrun image2_iris.cfg association.json

## Plot the reduced frame

Finally we can use `iris_pipeline` to load the data and plot them:

In [ ]:
import iris_pipeline
reduced_science_frame = iris_pipeline.datamodels.IRISImageModel("test_iris_subtract_bg_flat_cal.fits")
try:    
    %matplotlib inline
    import matplotlib.pyplot as plt
    plt.figure()
    plt.imshow(reduced_science_frame.data, vmin=0, vmax=1500)
    plt.title("Processed frame")

except ImportError:
    pass